# Course AI Homework 5
In Homework 5, we will train our own 'CBOW' Word2Vec embedding from WikiText2 dataset. (small dataset)
- Change Runtime option above to GPU if you could. (max 12 hours for one user)
- Save and submit the outputs of this notebook and model and vocab file you trained.
- Not allowed to have other python file or import pretrained model.

In [1]:
# YOU should run this command if you will train the model in COLAB environment
! pip install datasets transformers torchtext==0.17.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16

In [22]:
import argparse
import yaml
import os
import torch
import torch.nn as nn
import torchtext

import json
import numpy as np

from functools import partial
from torch.utils.data import DataLoader
from torchtext.data import to_map_style_dataset
from torchtext.data.utils import get_tokenizer

from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import WikiText2 # WikiText103

import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR

from datasets import load_dataset



In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch_seed_numb = 0
if device.type == 'cuda':
    torch.cuda.manual_seed(torch_seed_numb)

In [4]:
device

device(type='cuda')

In [5]:
# If you use Google Colab environment, mount you google drive here to save model and vocab
from google.colab import drive
drive.mount('/content/drive')
root_dir = '/content/drive/MyDrive/course_ai_hw5'

Mounted at /content/drive


### Constant Setting

In [6]:
# You could change parameters if you want.

train_batch_size =  96
val_batch_size = 96
shuffle =  True

optimizer =  'Adam'
learning_rate =  0.025
epochs = 50

result_dir = 'weights/'

# Parameters about CBOW model architecture and Vocab.
CBOW_N_WORDS = 4

MIN_WORD_FREQUENCY = 50
MAX_SEQUENCE_LENGTH = 256

EMBED_DIMENSION = 300
EMBED_MAX_NORM = 1

In [7]:
result_dir = os.path.join(root_dir, result_dir)
if not os.path.exists(result_dir):
    os.makedirs(result_dir)


## Prepare dataset and vocab

In [8]:
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')
train_dataset = datasets["train"]
val_dataset = datasets['validation']
test_dataset = datasets['test']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [9]:
# Let's print one example
train_dataset['text'][11]

" Troops are divided into five classes : Scouts , Shocktroopers , Engineers , Lancers and Armored Soldier . Troopers can switch classes by changing their assigned weapon . Changing class does not greatly affect the stats gained while in a previous class . With victory in battle , experience points are awarded to the squad , which are distributed into five different attributes shared by the entire squad , a feature differing from early games ' method of distributing to different unit types . \n"

As you can see, we need to clean up the sentences, lowercase them, tokenize them, and change each word into an index (one-hot vector). Before going through the whole process, we need to create a vocab set using the training dataset.

In [34]:
tokenizer = get_tokenizer("basic_english", language="en")

# TODO 1): make vocabulary
# Hint) use function: build_vocab_from_iterator, use train_dataset set special tokens.. etc
def yield_tokens(data_iter):
    for data in data_iter:
        yield tokenizer(data)

vocab = build_vocab_from_iterator(yield_tokens(train_dataset['text']), specials=['<unk>'], min_freq=MIN_WORD_FREQUENCY)
vocab.set_default_index(vocab['<unk>'])

We need a collate function to make dataset into CBOW trainning format. The collate function should iterate over (sliding) batch data and make train/test dataset.And each component of data should be composed of CBOW_N_WORD words in each left and right side as input and target output as word in center.  
Make the collate function return CBOW dataset in tensor type.

In [33]:
# Here is a lambda function to tokenize sentence and change words to vocab indexes.
text_pipeline = lambda x: vocab(tokenizer(x))

![cbow](https://user-images.githubusercontent.com/74028313/204695601-51d44a38-4bd3-4a69-8891-2854aa57c034.png)

In [55]:
def collate(batch, text_pipeline):

    batch_input, batch_output = [], []

    # TODO 2): make collate function
    for data in batch:
        text = text_pipeline(data)
        for i in range(0, len(text) - 2*CBOW_N_WORDS):
            context1 = text[i:i+CBOW_N_WORDS]
            target = text[i+CBOW_N_WORDS]
            context2 = text[i+1+CBOW_N_WORDS:i+CBOW_N_WORDS*2+1]
            batch_input.append(context1 + context2)
            batch_output.append(target)

    return batch_input, batch_output

In [56]:
train_dataloader = DataLoader(
    train_dataset['text'],
    batch_size=train_batch_size,
    shuffle=shuffle,
    collate_fn=partial(collate, text_pipeline=text_pipeline),
)

val_dataloader = DataLoader(
    val_dataset['text'],
    batch_size=val_batch_size,
    shuffle=shuffle,
    collate_fn=partial(collate, text_pipeline=text_pipeline),
)

## Make CBOW Model
![image](https://user-images.githubusercontent.com/74028313/204701161-cd9df4bf-78b8-4b4d-b8b7-ed4a3b5c3922.png)

CBOW Models' main concept is to predict center-target word using context words. As you see in above simple architecture, input 2XCBOW_N_WORDS length words are projected to Projection layer. In order to convert each word to embedding, it needs look-up table and we will use torch's Embedding function to convert it. After combining embeddings of context, it use shallow linear neural network to predict target word and compare result with center word's index using cross-entropy loss. Finally, the embedding layer (lookup table) of the trained model itself serves as an embedding representing words.

In [38]:
class CBOW_Model(nn.Module):
    def __init__(self, vocab_size: int, EMBED_DIMENSION, EMBED_MAX_NORM):
        super(CBOW_Model, self).__init__()
        # TODO 3-1): make CBOW model using nn.Embedding and nn.Linear function
        self.embedding = nn.Embedding(vocab_size, EMBED_DIMENSION)
        self.fc = nn.Linear(EMBED_DIMENSION, vocab_size)

    def forward(self, _inputs):
        # TODO 3-2): make forward function
        _inputs = self.embedding(_inputs)
        _inputs = _inputs.mean(dim=1)
        _outputs = self.fc(_inputs)

        return _outputs

## Train the model

Let's make _train_epoch and _validate_epoch functions to train the CBOW model.  
- model.train() and model.eval() change torch mode in some parts (Dropout, BatchNorm..  etc) of the model to behave differently during inference time.
- There is lr_scheduler option which changes learning rate according to epoch level. Try the option if you are interested in.

In [39]:
vocab_size = len(vocab.get_stoi())

model = CBOW_Model(vocab_size=vocab_size, EMBED_DIMENSION = EMBED_DIMENSION, EMBED_MAX_NORM = EMBED_MAX_NORM)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [41]:
class Train_CBOW:

    def __init__(
        self,
        model,
        epochs,
        train_dataloader,
        val_dataloader,
        loss_function,
        optimizer,
        device,
        model_dir,
        lr_scheduler = None
    ):
        self.model = model
        self.epochs = epochs
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.loss_function = loss_function
        self.optimizer = optimizer
        self.lr_scheduler = lr_scheduler
        self.device = device
        self.model_dir = model_dir

        self.loss = {"train": [], "val": []}
        self.model.to(self.device)

    def train(self):
        for epoch in range(self.epochs):
            self._train_epoch()
            self._validate_epoch()
            print(
                "Epoch: {}/{}, Train Loss={:.5f}, Val Loss={:.5f}".format(
                    epoch + 1,
                    self.epochs,
                    self.loss["train"][-1],
                    self.loss["val"][-1],
                )
            )
            if self.lr_scheduler is not None:
                self.lr_scheduler.step()


    def _train_epoch(self):
        self.model.train() # set model as train
        loss_list = []
        # TODO 4-1):
        for batch in self.train_dataloader:
            self.optimizer.zero_grad()
            inputs, targets = batch
            inputs, targets = torch.tensor(inputs).to(self.device), torch.tensor(targets).to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_function(outputs, targets)
            loss.backward()
            self.optimizer.step()
            loss_list.append(loss.item())

        # end of TODO
        epoch_loss = np.mean(loss_list)
        self.loss["train"].append(epoch_loss)

    def _validate_epoch(self):
        self.model.eval()
        loss_list = []

        with torch.no_grad():
            # TODO 4-2):
            for batch in self.val_dataloader:
                inputs, targets = batch
                inputs, targets = torch.tensor(inputs).to(self.device), torch.tensor(targets).to(self.device)
                outputs = self.model(inputs)
                loss = self.loss_function(outputs, targets)
                loss_list.append(loss.item())
            # end of TODO
        epoch_loss = np.mean(loss_list)
        self.loss["val"].append(epoch_loss)


    def save_model(self):
        model_path = os.path.join(self.model_dir, "model.pt")
        torch.save(self.model, model_path)

    def save_loss(self):
        loss_path = os.path.join(self.model_dir, "loss.json")
        with open(loss_path, "w") as fp:
            json.dump(self.loss, fp)

In [42]:
# Option: you could add and change lr_sceduler
scheduler = LambdaLR(optimizer, lr_lambda = lambda epoch: 0.95 ** epoch)

In [57]:
trainer = Train_CBOW(
    model=model,
    epochs=epochs,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    loss_function=loss_function,
    optimizer=optimizer,
    lr_scheduler=None,
    device=device,
    model_dir=result_dir,
)

trainer.train()
print("Training finished.")


Epoch: 1/50, Train Loss=4.95590, Val Loss=4.75036
Epoch: 2/50, Train Loss=4.45806, Val Loss=4.79600
Epoch: 3/50, Train Loss=4.30840, Val Loss=4.83062
Epoch: 4/50, Train Loss=4.22109, Val Loss=4.89513
Epoch: 5/50, Train Loss=4.16405, Val Loss=4.95069
Epoch: 6/50, Train Loss=4.12205, Val Loss=5.00348
Epoch: 7/50, Train Loss=4.09729, Val Loss=5.03114
Epoch: 8/50, Train Loss=4.07385, Val Loss=5.05604
Epoch: 9/50, Train Loss=4.05670, Val Loss=5.08601
Epoch: 10/50, Train Loss=4.04184, Val Loss=5.08772
Epoch: 11/50, Train Loss=4.02936, Val Loss=5.13148
Epoch: 12/50, Train Loss=4.02146, Val Loss=5.12856
Epoch: 13/50, Train Loss=4.00778, Val Loss=5.15027
Epoch: 14/50, Train Loss=4.00633, Val Loss=5.15849
Epoch: 15/50, Train Loss=3.99497, Val Loss=5.18152
Epoch: 16/50, Train Loss=3.99450, Val Loss=5.19676
Epoch: 17/50, Train Loss=3.98791, Val Loss=5.19822
Epoch: 18/50, Train Loss=3.97919, Val Loss=5.22064
Epoch: 19/50, Train Loss=3.98068, Val Loss=5.19525
Epoch: 20/50, Train Loss=3.97210, Val Lo

In [58]:
# save model
trainer.save_model()
trainer.save_loss()

vocab_path = os.path.join(result_dir, "vocab.pt")
torch.save(vocab, vocab_path)

### Result
Let's inference trained word embedding and visualize it.

In [59]:
import pandas as pd
import sys

from sklearn.manifold import TSNE
import plotly.graph_objects as go

sys.path.append("../")

In [60]:
result_dir

'/content/drive/MyDrive/course_ai_hw5/weights/'

In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# reload saved model and vocab
model = torch.load(os.path.join(result_dir,"model.pt"), map_location=device)
vocab = torch.load(os.path.join(result_dir,"vocab.pt"))

# embedding is model's first layer
embeddings = list(model.parameters())[0]
embeddings = embeddings.cpu().detach().numpy()

# normalization
norms = (embeddings ** 2).sum(axis=1) ** (1 / 2)
norms = np.reshape(norms, (len(norms), 1))
embeddings_norm = embeddings / norms
embeddings_norm.shape



(4119, 300)

### Make t-SNE graph of trained embedding and color numeric values

In [67]:
embeddings_df = pd.DataFrame(embeddings_norm)
fig = go.Figure()
# TODO 5-1) : make 2-d t-SNE graph of all vocabs and color only for numeric values(others, just color black)
tsne = TSNE(n_components=2)
embeddings_tsne = tsne.fit_transform(embeddings_df)
embeddings_tsne_df = pd.DataFrame(embeddings_tsne, columns=["x", "y"])
embeddings_tsne_df["vocab"] = vocab.get_itos()
embeddings_tsne_df["is_numeric"] = embeddings_tsne_df["vocab"].str.isnumeric()

for is_numeric, group in embeddings_tsne_df.groupby("is_numeric"):
    color = "red" if is_numeric else "black"
    fig.add_trace(
        go.Scatter(
            x=group["x"],
            y=group["y"],
            mode="markers+text",
            text=group["vocab"],
            textposition="top center",
            marker=dict(color=color),
            name="Numeric" if is_numeric else "Non-numeric"
        )
    )

fig.update_layout(title="t-SNE visualization of word embeddings")
fig.show()

### Find top N similar words


In [63]:
def find_top_similar(word: str, vocab, embeddings_norm, topN: int = 10):
    # TODO 5-2) : make function returning top n similiar words and similarity scores
    topN_dict = {}
    word_idx = vocab[word]
    word_vector = embeddings_norm[word_idx]
    for idx, vector in enumerate(embeddings_norm):
        if idx == word_idx:
            continue
        similarity = np.dot(word_vector, vector)
        topN_dict[vocab.get_itos()[idx]] = similarity
    topN_dict = dict(sorted(topN_dict.items(), key=lambda x: x[1], reverse=True)[:topN])
    return topN_dict


In [64]:
for word, sim in find_top_similar("english", vocab, embeddings_norm).items():
    print("{}: {:.3f}".format(word, sim))


assumed: 0.240
america: 0.202
thick: 0.197
adult: 0.195
bible: 0.193
oldest: 0.188
floor: 0.182
140: 0.178
1940s: 0.177
amateur: 0.174


### Result Report

Save the colab result and submit it with your trained model file, vocab file, and t-SNE result image in the .zip format. Check one more time your submitted notebook file has result.

You can change the CBOW model parameters Training parameters and details if you want.